In [32]:
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
import pandas as pd

from k_tsp_solver import (
    Experiment,
    ModelName,
    KFactor,
    logger
)

In [29]:
TSP_SELECTED_INSTANCES = [
    "gr24.tsp", "berlin52.tsp", "pr76.tsp",
    "ch130.tsp", "pr152.tsp", "kroA200.tsp",
    "att532.tsp", "rat783.tsp"
]

ATSP_SELECTED_INSTANCES = [
    "br17.atsp", "ftv170.atsp", "rbg403.atsp"
]

SELECTED_INSTANCES = TSP_SELECTED_INSTANCES + ATSP_SELECTED_INSTANCES

In [24]:
fixed_k = KFactor.LARGE
model_name = ModelName.GENETIC_ALGORITHM_NEAREST_NEIGHBORS_ENSEMBLE.value

search_space = [
    Integer(50, 250, name="population_size"),
    Integer(50, 250, name="generations"),
    Real(0.01, 0.1, name="mutation_rate"),
    Integer(5, 40, name="selection_size"),
    Categorical([True, False], name="has_variable_mutate_rate")
]

In [35]:
def objective(params):
    population_size, generations, mutation_rate, selection_size, has_var = params
    params_dict = {
        "population_size": population_size,
        "generations": generations,
        "mutation_rate": mutation_rate,
        "selection_size": selection_size,
        "has_variable_mutate_rate": has_var,
        "is_debugging": False
    }

    results = []
    for instance in ["gr24.tsp"]:
        for has_closed in [False, True]:
            logger.info(f"Running {model_name} on {instance} with params: {params_dict}")
            exp = Experiment(
                instance_name=instance,
                model_name=model_name,
                model_parameters=params_dict,
                k_factor=fixed_k,
                has_closed_cycle=has_closed,
                repetitions=15
            )
            exp.run()
            best_length = exp._get_experiment_as_dataframe()["path_length"].min()
            results.append(best_length)

    return float(pd.Series(results).mean())

In [ ]:
search_result = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=10,
    random_state=0,
    verbose=True
)

In [38]:
best_params = dict(zip(
    ["population_size", "generations", "mutation_rate", "selection_size", "has_variable_mutate_rate"],
    search_result.x
))
best_params

{'population_size': np.int64(80),
 'generations': np.int64(94),
 'mutation_rate': 0.04478400830132759,
 'selection_size': np.int64(37),
 'has_variable_mutate_rate': True}

In [40]:
convergence = pd.DataFrame({
    "iteration": list(range(1, len(search_result.func_vals) + 1)),
    "mean_best_length": search_result.func_vals
})

In [41]:
convergence

,iteration,mean_best_length
0,1,795.0
1,2,793.5
2,3,789.5
3,4,781.5
4,5,795.5
5,6,795.5
6,7,795.5
7,8,776.5
8,9,791.0
9,10,776.5
